# Analysis code for the paper "B1+ shimming in the cervical spinal cord at 7T"

## Data

The data can be download at: XXX (TODO: upload on openneuro)

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):
~~~
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~


## Overview of processing pipeline

For each subject:

- Segment the SC from the GRE scan
- Extract the signal intensity within this mask from the GRE scans
- Coregister the TFL_B1map to the corresponding GRE scan from each subfolder (CVred, Noshim, etc.)
- Apply the computed warping field to bring the SC segmentation to the TFL_B1map
- Convert the B1+ maps to nT/V units
- Extract the B1+ value within the registered mask

In [ ]:
# Necessary imports

import os
import re
import json
import subprocess
import glob
import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# from IPython.display import display
# from tabulate import tabulate
import nibabel as nib
# import pandas as pd

In [ ]:
# Download data and define path variables

!datalad install https://github.com/OpenNeuroDatasets/ds004906.git
os.chdir("ds004906")
# !datalad get . # uncomment for production
!datalad get sub-01/  # comment for debugging

In [ ]:
# Define useful variables

path_data = os.getcwd()
path_qc = os.path.join(path_data, "qc")
shim_modes = ["CP", "CoV", "patient", "phase", "SAReff", "target", "volume"]
subjects = glob.glob("sub-*")

In [ ]:
# Run segmentation on GRE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2 -qc {path_qc}

## Verify quality control (QC) report

Open the QC report located under `ds004906/qc/index.html`. Make sure the segmentations and registrations are correct before resuming the analysis.

In [ ]:
# Crop GRE scan for faster processing and better registration results

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_crop_image -i {subject}_acq-CoV_T2starw.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop.nii.gz
    !sct_crop_image -i {subject}_acq-CoV_T2starw_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_seg.nii.gz

In [ ]:
# Register TFL B1maps to the GRE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_register_multimodal -i {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -d ../anat/{subject}_acq-CoV_T2starw_crop.nii.gz -dseg ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {path_qc}